# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
from IPython.display import HTML
import gmaps.datasets
from ipywidgets.embed import embed_minimal_html

# Import API key
from api_keys import g_key



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv("../WeatherPy/cities.csv", index_col=False)
cities_df

,Max Temp,Humidity,Lat,Lng,Cloudiness,Country,City,Wind Speed,Date,parsed_time
0,83.28,78,-0.60,73.08,79,MV,Hithadhoo,14.74,1579924152,2020-01-25 03:49:12
1,66.20,82,-33.92,25.57,62,ZA,Port Elizabeth,10.29,1579923904,2020-01-25 03:45:04
2,-7.60,76,65.28,-126.83,90,CA,Norman Wells,3.36,1579923979,2020-01-25 03:46:19
3,45.00,93,42.10,-79.24,90,US,Jamestown,9.17,1579924101,2020-01-25 03:48:21
4,78.80,78,13.22,-59.52,0,BB,Bathsheba,8.05,1579924156,2020-01-25 03:49:16
...,...,...,...,...,...,...,...,...,...,...
539,6.80,84,60.82,-115.80,75,CA,Hay River,5.82,1579924805,2020-01-25 04:00:05
540,12.20,85,64.75,177.48,90,RU,Anadyr,6.71,1579924779,2020-01-25 03:59:39
541,5.47,97,66.08,33.04,56,RU,Loukhi,4.41,1579924807,2020-01-25 04:00:07
542,79.66,74,-20.23,164.02,19,NC,Poum,13.42,1579924747,2020-01-25 03:59:07


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [10]:
gmaps.configure(api_key=g_key)

figure_layout = {
'width': '800px',
'height': '600px',
'border': '1px solid black',
'padding': '1px'
}
#
fig = gmaps.figure(layout=figure_layout, zoom_level=1.6, center=[41.9028,12.4964])
heatmap_layer = gmaps.heatmap_layer(
    cities_df[['Lat', 'Lng']], weights=cities_df['Humidity'],
    max_intensity=30, point_radius=5.0)
fig.add_layer(heatmap_layer)
fig
#embed_minimal_html('map.html', views=[fig])
#HTML(filename="./map.html")

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='1px', width='800px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

Favorable weather conditions for summer vacation: 

* A max temperature lower than 80 degrees but higher than 70.

* Wind speed less than 10 mph.

* Zero cloudiness.


In [4]:
#null and nan variable were not included in this database from the beginning.

cloudiness_mask = cities_df["Cloudiness"] <= 10
windSpeed_mask = cities_df['Wind Speed'] < 10
maxTemp_mask = (cities_df['Max Temp'] > 70) & (cities_df['Max Temp'] < 80)


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
narrowed_city_df = cities_df[cloudiness_mask & windSpeed_mask & maxTemp_mask]
hotel_df = pd.DataFrame()

for index, row in narrowed_city_df.iterrows():
    try:
        url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={narrowed_city_df.loc[index, 'Lat']},{narrowed_city_df.loc[index,'Lng']}&radius=6000&type=lodging&keyword=hotel&key={g_key}"
        response = requests.get(url).json()
        hotel_df.loc[index, 'Hotel Name'] = response['results'][0]['name']
        hotel_df.loc[index, 'Lat'] = response['results'][0]['geometry']['location']['lat']
        hotel_df.loc[index, 'Lng'] = response['results'][0]['geometry']['location']['lng']
        hotel_df.loc[index, 'City'] = row['City']
        hotel_df.loc[index, 'Country'] = row['Country']
        print(f"Successfully fetched information about hotel near {row['City']}")
    except:
        print(f"Information about hotel near {row['City']} could not be fetched")

Successfully fetched information about hotel near Bathsheba
Successfully fetched information about hotel near Sassandra
Successfully fetched information about hotel near Cidreira
Successfully fetched information about hotel near Yenagoa
Successfully fetched information about hotel near Oistins
Information about hotel near Lagos could not be fetched
Information about hotel near Rosario de Mora could not be fetched
Successfully fetched information about hotel near Zwedru
Successfully fetched information about hotel near Harper
Successfully fetched information about hotel near Makakilo City
Successfully fetched information about hotel near Colorado
Successfully fetched information about hotel near Rocha
Successfully fetched information about hotel near Rondonopolis
Successfully fetched information about hotel near Champerico
Successfully fetched information about hotel near Fuerte Olimpo
Successfully fetched information about hotel near Mercedes
Successfully fetched information about hote

In [6]:
hotel_df = hotel_df.reset_index(drop=True)
hotel_df

,Hotel Name,Lat,Lng,City,Country
0,ECO Lifestyle & Lodge,13.210992,-59.515715,Bathsheba,BB
1,JOYAU CACHÉ,4.946397,-6.085625,Sassandra,CI
2,Pousada Rosa,-30.192029,-50.209673,Cidreira,BR
3,144 SUITES LUXURY HOTEL,4.936638,6.316138,Yenagoa,NG
4,Chateau Blanc Apartments on Sea,13.070498,-59.580747,Oistins,BB
5,B and P Guest House,6.067688,-8.141417,Zwedru,LR
6,Grands' Guesthouse,4.387830,-7.692412,Harper,LR
7,Marriott's Ko Olina Beach Club,21.333045,-158.120239,Makakilo City,US
8,Sevilha Park Hotel,-22.831110,-51.971541,Colorado,BR
9,Casa Rodo 53,-34.483518,-54.335788,Rocha,UY


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template ="""
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
hotel_info

['\n<dl>\n<dt>Name</dt><dd>ECO Lifestyle & Lodge</dd>\n<dt>City</dt><dd>Bathsheba</dd>\n<dt>Country</dt><dd>BB</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>JOYAU CACHÉ</dd>\n<dt>City</dt><dd>Sassandra</dd>\n<dt>Country</dt><dd>CI</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Pousada Rosa</dd>\n<dt>City</dt><dd>Cidreira</dd>\n<dt>Country</dt><dd>BR</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>144 SUITES LUXURY HOTEL</dd>\n<dt>City</dt><dd>Yenagoa</dd>\n<dt>Country</dt><dd>NG</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Chateau Blanc Apartments on Sea</dd>\n<dt>City</dt><dd>Oistins</dd>\n<dt>Country</dt><dd>BB</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>B and P Guest House</dd>\n<dt>City</dt><dd>Zwedru</dd>\n<dt>Country</dt><dd>LR</dd>\n</dl>\n',
 "\n<dl>\n<dt>Name</dt><dd>Grands' Guesthouse</dd>\n<dt>City</dt><dd>Harper</dd>\n<dt>Country</dt><dd>LR</dd>\n</dl>\n",
 "\n<dl>\n<dt>Name</dt><dd>Marriott's Ko Olina Beach Club</dd>\n<dt>City</dt><dd>Makakilo City</dd>\n<dt>Country</dt><dd>US</dd>\n</dl

In [9]:
# Add marker layer ontop of heat map


markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig
# Display Map
#embed_minimal_html('vacation_map.html', views=[fig])
#HTML(filename="./vacation_map.html")

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='1px', width='800px'))